In [71]:
# using Pkg
# Pkg.add("JuMP")
# Pkg.add("Gurobi")
# Pkg.add("Distances")
# Pkg.add("Distributions")
# Pkg.add("DataFrames")

In [72]:
using JuMP
using Gurobi
using DataFrames
using Random

# Generate Data

## Constants

In [73]:
warehouse_location_side_size = 2
warehouse_rows = 20
warehouse_columns = 50

time_horizon = 20
num_shipments = 100

max_item_size = 10

S = num_shipments
L = warehouse_rows * warehouse_columns
T = time_horizon

20

In [74]:
Random.seed!(3000)

arrival_location = (0, 1)
departure_location = (warehouse_rows, 0)

distance_to_arrival = zeros(warehouse_rows * warehouse_columns)
for i in 1:warehouse_rows, j in 1:warehouse_columns
    l = (i-1)*warehouse_columns + j
    distance_to_arrival[l] = i * warehouse_location_side_size + j * warehouse_location_side_size
end

distance_to_departure = zeros(warehouse_rows * warehouse_columns)
for i in 1:warehouse_rows, j in 1:warehouse_columns
    l = (i-1)*warehouse_columns + j
    distance_to_departure[l] = ((warehouse_rows - i) * warehouse_location_side_size) + j * warehouse_location_side_size
end

location_max_size = rand(1:max_item_size, warehouse_rows * warehouse_columns)

rand_shipment_data = rand(1:T, (S, 2))
arrivals = minimum(rand_shipment_data, dims=2)
departures = maximum(rand_shipment_data, dims=2)

rand_sizes = rand(1:max_item_size, S)

struct Shipment
    arrival_time::Int
    departure_time::Int
    product_size::Int
end

shipments = Array{Shipment,1}(undef, S)

for s in 1:S
    arrival_time = arrivals[s]
    departure_time = departures[s]
    product_size = rand_sizes[s]

    # Can't arrive and depart at the same time
    if arrival_time == departure_time
        if departure_time == T
            arrival_time -= 1
        else
            departure_time += 1
        end
    end

    shipments[s] = Shipment(arrival_time, departure_time, product_size)
end

# Build Model

In [75]:
# model = Model(Gurobi.Optimizer)
# set_optimizer_attribute(model, "TimeLimit", 60);

# # Are we assigning the product from shipment s to location l at time t
# @variable(
#     model,
#     r[1:S,1:L,1:T] >= 0, Bin
# ) ;

# # Are we assigning the product from shipment s to location l at time t
# @variable(
#     model,
#     z[1:S,1:L] >= 0, Bin
# ) ;

# @objective(
#     model,
#     Min,
#     sum(
#         (distance_to_arrival[l] + distance_to_departure[l]) * z[s,l]
#         for s in 1:S,
#             l in 1:L
#     )
# );

# # @objective(
# #     model,
# #     Min,
# #     sum(
# #         (distance_to_arrival[l] + distance_to_departure[l]) / (shipments[s].departure_time - shipments[s].arrival_time) * r[s,l,t]
# #         for s in 1:S,
# #             l in 1:L,
# #             t in 1:T
# #     )
# # );

# # Within its window, a product is stored in its location and no where else
# for s in 1:S
#     for l in 1:L
#         for t in shipments[s].arrival_time:shipments[s].departure_time
#             @constraint(
#                 model,
#                 r[s,l,t] == z[s,l]
#             );
#         end
#     end
# end

# # Each product must be assigned to exactly one location during its time window:
# for s in 1:S
#     for t in shipments[s].arrival_time:shipments[s].departure_time
#         @constraint(
#             model,
#             sum(
#                 r[s,l,t]
#                 for l in 1:L
#             ) == 1
#         );
#     end
# end

# # Each product is only assigned to a location during its time window:
# for s in 1:S
#     for t in 1:shipments[s].arrival_time-1
#         @constraint(
#             model,
#             sum(
#                 r[s,l,t]
#                 for l in 1:L
#             ) == 0
#         );
#     end
# end
# for s in 1:S
#     for t in shipments[s].departure_time+1:T
#         @constraint(
#             model,
#             sum(
#                 r[s,l,t]
#                 for l in 1:L
#             ) == 0
#         );
#     end
# end

# # Each location can hold at most one product at any specific time:
# for l in 1:L
#     for t in 1:T
#         @constraint(
#             model,
#             sum(
#                 r[s,l,t]
#                 for s in 1:S
#             ) <= 1
#         );
#     end
# end

# # Each location can only hold items up to max size of the location
# for l in 1:L
#     for t in 1:T
#         for s in 1:S
#             @constraint(
#                 model,
#                 r[s,l,t] * shipments[s].product_size <= location_max_size[l]
#             );
#         end
#     end
# end

In [85]:
# function overlap(A1::Int, D1::Int, A2::Int, D2::Int)
#     if A1 <= D2 && A2 <= D1
#         return true
#     else
#         return false
#     end
# end

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 60);

# Are we assigning the product from shipment s to location l at time t
# @variable(
#     model,
#     r[1:S,1:L,1:T] >= 0, Bin
# ) ;

# Are we assigning the product from shipment s to location l at time t
@variable(
    model,
    1 <= z[1:S] <= L, Int
) ;

@objective(
    model,
    Min,
    sum(
        # distance_to_arrival[z[s]] + distance_to_departure[z[s]]
        z[s]
        for s in 1:S
    )
);

# @objective(
#     model,
#     Min,
#     sum(
#         (distance_to_arrival[l] + distance_to_departure[l]) / (shipments[s].departure_time - shipments[s].arrival_time) * r[s,l,t]
#         for s in 1:S,
#             l in 1:L,
#             t in 1:T
#     )
# );

# Within its window, a product is stored in its location and no where else
# for s in 1:S
#     for l in 1:L
#         for t in shipments[s].arrival_time:shipments[s].departure_time
#             @constraint(
#                 model,
#                 r[s,l,t] == z[s,l]
#             );
#         end
#     end
# end

# Each product must be assigned to exactly one location during its time window:
# for s in 1:S
#     for t in shipments[s].arrival_time:shipments[s].departure_time
#         @constraint(
#             model,
#             sum(
#                 r[s,l,t]
#                 for l in 1:L
#             ) == 1
#         );
#     end
# end

# Each product is only assigned to a location during its time window:
# for s in 1:S
#     for t in 1:shipments[s].arrival_time-1
#         @constraint(
#             model,
#             sum(
#                 r[s,l,t]
#                 for l in 1:L
#             ) == 0
#         );
#     end
# end
# for s in 1:S
#     for t in shipments[s].departure_time+1:T
#         @constraint(
#             model,
#             sum(
#                 r[s,l,t]
#                 for l in 1:L
#             ) == 0
#         );
#     end
# end

# Each location can hold at most one product at any specific time:
for s1 in 1:S
    for s2 in 1:S
        if shipments[s1].arrival_time <= shipments[s2].departure_time && shipments[s2].arrival_time <= shipments[s1].departure_time
            z[s1] != z[s2]
        # @constraint(
        #     model,
        #     sum(
        #         r[s,l,t]
        #         for s in 1:S
        #     ) <= 1
        # );
        end
    end
end

# Each location can only hold items up to max size of the location
# for l in 1:L
for s in 1:S
    @constraint(
        model,
        z[s] * shipments[s].product_size <= location_max_size[z[s]]
    );
end
# end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-08
Set parameter TimeLimit to value 60


In [86]:
optimize!(model)

Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 0 rows, 100 columns and 0 nonzeros
Model fingerprint: 0xbd48a2bb
Variable types: 0 continuous, 100 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 100.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 1: 100 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+02, best bound 1.000000000000e+02, gap 0.0000%

User-callback calls 24, time in user-callback 0.00 sec


In [78]:
# value.(r[1,:,:])

findall(x->x==1, value.(r[1,:,:]))

13-element Vector{CartesianIndex{2}}:
 CartesianIndex(403, 7)
 CartesianIndex(403, 8)
 CartesianIndex(403, 9)
 CartesianIndex(403, 10)
 CartesianIndex(403, 11)
 CartesianIndex(403, 12)
 CartesianIndex(403, 13)
 CartesianIndex(403, 14)
 CartesianIndex(403, 15)
 CartesianIndex(403, 16)
 CartesianIndex(403, 17)
 CartesianIndex(403, 18)
 CartesianIndex(403, 19)